<a href="https://colab.research.google.com/github/ambrosioo/Dashboards/blob/main/dados_itens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import timedelta

df = pd.read_excel('relatorio_de_itens.xlsx')

feriados = [
    '2024-01-01',
    '2024-02-12',
    '2024-02-13',
    '2024-02-14',
    '2024-01-25'
]

df['Aberto'] = pd.to_datetime(df['Aberto'])

df['Tempo de resolução'] = pd.to_datetime(df['Tempo de resolução'], errors='coerce')

def is_dia_util(data):
    if data.weekday() >= 5:
        return False
    if data.strftime('%Y-%m-%d') in feriados:
        return False
    return True

def dias_uteis_entre_datas(data_inicio, data_fim):
    if pd.isnull(data_fim):
        return pd.NaT
    dias_uteis = 0
    for i in range((data_fim - data_inicio).days + 1):
        dia = data_inicio + timedelta(days=i)
        if is_dia_util(dia):
            dias_uteis += 1
    return dias_uteis

df['Tempo de resolução_dias_uteis'] = df.apply(lambda row: dias_uteis_entre_datas(row['Aberto'], row['Tempo de resolução']), axis=1)

def sla_atendido(tipo_solicitacao, dias_uteis):
    if pd.isnull(dias_uteis):
        return ""

    sla = {
        "Ajustar um item": 8,
        "Copiar um item": 3,
        "Criar um item": 8,
        "Expandir um item": 5
    }
    if dias_uteis > sla[tipo_solicitacao]:
        return "out"
    else:
        return "in"

df['SLA IN/OUT'] = df.apply(lambda row: sla_atendido(row['Qual tipo de Solicitação?'], row['Tempo de resolução_dias_uteis']), axis=1)

print(df)


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


       Número              Aberto     Estado                       Item  \
0   SR0239664 2024-03-20 09:47:53     Aberto  Itens de Compras | Oracle   
1   SR0239525 2024-03-20 08:12:40     Aberto  Itens de Compras | Oracle   
2   SR0238944 2024-03-18 15:56:20  Resolvido  Itens de Compras | Oracle   
3   SR0238940 2024-03-18 15:53:46     Aberto  Itens de Compras | Oracle   
4   SR0238935 2024-03-18 15:51:17  Resolvido  Itens de Compras | Oracle   
..        ...                 ...        ...                        ...   
83  SR0220111 2024-01-10 16:07:17    Fechado  Itens de Compras | Oracle   
84  SR0220110 2024-01-10 16:06:28    Fechado  Itens de Compras | Oracle   
85  SR0220041 2024-01-10 14:06:03  Cancelado  Itens de Compras | Oracle   
86  SR0220013 2024-01-10 13:07:25    Fechado  Itens de Compras | Oracle   
87  SR0219782 2024-01-09 16:18:12    Fechado  Itens de Compras | Oracle   

    Tarefa de entrega  Tarefa para  \
0                 NaN          NaN   
1                 NaN  

In [ ]:
df.to_excel('resultado_analise.xlsx', index=False)


In [ ]:
df.to_csv('resultado_analise.csv', index=False)